# Project: Classifying buildings post-Hurricane
Univ.AI <br>
AI-2 Cohort-4

## [DATASET](https://drive.google.com/file/d/1EH3p84xKMs_2m4ISSR7aHOQz61yxIv8L/view?usp=sharing)

### Names of the people who worked on the project:

Link to presentation: <br>
Link to slides: <br>
Link to poster: <br>
Link to notebook: <br>

In [1]:
import os
import certifi
import urllib3  # For handling https certificate verification 
import requests
import zipfile
import shutil
import json
import time
import sys
import cv2
import numpy as np
import pandas as pd
from scipy.signal import convolve2d
import scipy.ndimage as ndimage
import subprocess
from PIL import Image
from matplotlib import cm
from matplotlib import pyplot
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import keras
import os, shutil
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.preprocessing import image
from keras.layers import LeakyReLU
from keras.regularizers import l2
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator




2022-12-25 20:45:54.586047: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-25 20:45:56.161650: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/munch/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-12-25 20:45:56.161782: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/munch/.local/lib/python3.10/site-packages/cv2/../../lib64:


In [3]:

train_damage_dir = 'data/train_another/damage'
validation_damage_dir = 'data/validation_another/damage'
test_damage_dir = 'data/test/damage'
test_another_damage_dir = 'data/test_another/damage'


train_nodamage_dir = 'data/train_another/no_damage'
validation_nodamage_dir = 'data/validation_another/no_damage'
test_nodamage_dir = 'data/test/no_damage'
test_another_nodamage_dir = 'data/test_another/no_damage'

In [4]:
# we need to create a convolutional neural netowrk to calssify a series of satellite images of cyclone damaged buildings into
# damged and non-damged. these are the only two classes we are interested in.
# use various strategies to improve the accuracy of the model, such as Adam, Drop out, Batch Normalization
# also use max pooling appropriately in the network
# the input shape of the images is 150x150x3




model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation = 'relu', input_shape = (150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.summary() 



2022-12-25 20:46:12.456137: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

2022-12-25 20:46:12.507544: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-25 20:46:12.507615: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-25 20:46:12.508206: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-25 20:46:12.527050: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_nod

In [5]:
#compile the model, use the adam optimizer and binary cross entropy loss function
model.compile(loss = 'binary_crossentropy', optimizer = optimizers.Adam(lr = 1e-4), metrics = ['acc'])



/home/munch/.local/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [6]:

#normalize the image pixel value to be between 0 and 1
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                    'data/train_another', 
                    target_size = (150,150),
                    batch_size = 20,
                    class_mode = 'binary')
validation_generator = test_datagen.flow_from_directory(
                    'data/validation_another',
                    target_size = (150,150),
                    batch_size = 20,
                    class_mode = 'binary')

Found 10000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [7]:
history = model.fit_generator(
            train_generator,
            steps_per_epoch=100,
            epochs=50,
            validation_data=validation_generator,
            validation_steps=50)

/tmp/ipykernel_27563/687701678.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/50


2022-12-25 20:46:21.099009: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401


100/100 [==============================] - 37s 345ms/step - loss: 0.6792 - acc: 0.5625 - val_loss: 0.6227 - val_acc: 0.8210
Epoch 2/50
100/100 [==============================] - 29s 291ms/step - loss: 0.5267 - acc: 0.7625 - val_loss: 0.4087 - val_acc: 0.8310
Epoch 3/50
100/100 [==============================] - 30s 297ms/step - loss: 0.4391 - acc: 0.8180 - val_loss: 0.3945 - val_acc: 0.8490
Epoch 4/50
100/100 [==============================] - 29s 286ms/step - loss: 0.4117 - acc: 0.8265 - val_loss: 0.3725 - val_acc: 0.8550
Epoch 5/50
100/100 [==============================] - 28s 281ms/step - loss: 0.3953 - acc: 0.8460 - val_loss: 0.3428 - val_acc: 0.8670
Epoch 6/50
100/100 [==============================] - 29s 294ms/step - loss: 0.3402 - acc: 0.8650 - val_loss: 0.3234 - val_acc: 0.8830
Epoch 7/50
100/100 [==============================] - 28s 282ms/step - loss: 0.3540 - acc: 0.8575 - val_loss: 0.3470 - val_acc: 0.8620
Epoch 8/50
100/100 [==============================] - 30s 295ms/st

model.save('model.h5')

In [8]:
model.save('model_with_dropout.h5')